Raw CSV files acquired from (https://data.medicare.gov/data/inpatient-rehabilitation-facility-compare).

In [94]:
import pandas as pd

In [95]:
conditions = pd.read_csv('data/Inpatient_Rehab_Facilities_-_Revised_Flat_Files/Inpatient Rehab Facility - Conditions.csv')
general_info = pd.read_csv('data/Inpatient_Rehab_Facilities_-_Revised_Flat_Files/Inpatient Rehab Facility - General Information.csv')
provider_data = pd.read_csv('data/Inpatient_Rehab_Facilities_-_Revised_Flat_Files/Inpatient Rehab Facility - Provider data.csv')
national_data = pd.read_csv('data/Inpatient_Rehab_Facilities_-_Revised_Flat_Files/Inpatient Rehab Facility - National data.csv')

In [96]:
provider_data

,CMS Certification Number (CCN),Facility Name,Address Line 1,Address Line 2,City,State,Zip Code,County Name,PhoneNumber,CMS Region,Measure Code,Score,Footnote,Start Date,End Date
0,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_001_01_ADJ_RATE,0.7,NaN,04/01/2016,03/31/2017
1,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_001_01_DENOMINATOR,2259,NaN,04/01/2016,03/31/2017
2,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_002_01_DENOMINATOR,Not Available,4,10/01/2016,03/31/2017
3,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_002_01_OBS_RATE,Not Available,4,10/01/2016,03/31/2017
4,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_CI_LOWER,0.151,NaN,04/01/2016,03/31/2017
5,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_CI_UPPER,2.972,NaN,04/01/2016,03/31/2017
6,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_COMP_PERF,No Different than the National Benchmark,NaN,04/01/2016,03/31/2017
7,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_DOPC_DAYS,1568,NaN,04/01/2016,03/31/2017
8,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_ELIGCASES,2.223,NaN,04/01/2016,03/31/2017
9,013025,HEALTHSOUTH LAKESHORE REHABILITATION HOSPITAL,3800 RIDGEWAY DRIVE,NaN,BIRMINGHAM,AL,35209,Jefferson,(205) 868-2000,4,I_006_01_NUMERATOR,2,NaN,04/01/2016,03/31/2017


Let's start by finding out how many of the providers here are missing any data at all.

In [97]:
#Get total number of unique CCNs
total_ccns = len(provider_data["CMS Certification Number (CCN)"].unique())

#Check if there's any missing data for each CCN
missing_data_ccns = []
for ccn in provider_data["CMS Certification Number (CCN)"].unique():
    subset = provider_data.loc[provider_data["CMS Certification Number (CCN)"] == ccn]
    if subset["Score"].str.contains("Not Available").any():
        missing_data_ccns.append(ccn)
        
print("Number of facilities with missing data: ", len(missing_data_ccns))
print((float(len(missing_data_ccns)) / float(total_ccns)) * 100, "% of all facilities missing data.")

Number of facilities with missing data:  1192
100.0 % of all facilities missing data.


Well that's reassuring. Every single facility is missing a value for at least one field.

We can start by looking at individual metrics and moving from there. Let's start by looking at the rate of patient readmission.

In [98]:
#Select all rows that measure the number of stays and hte patient readmission rate
readmission_rates = provider_data.loc[
    (provider_data["Measure Code"] == "I_007_01_RSRR") | 
    (provider_data["Measure Code"] == "I_007_01_VOLUME")]

In [99]:
#Find all CCNs we want to filter out
filtered_ccns = []
for ccn in readmission_rates["CMS Certification Number (CCN)"].unique():
    #This is funky; just says if either of our two values for that CCN is "Not Available",
    #We'll drop that particular CCN altogether.
    if readmission_rates.loc[readmission_rates["CMS Certification Number (CCN)"] == ccn]["Score"].str.contains("Not Available").any():
        filtered_ccns.append(ccn)

        

print(len(readmission_rates.loc[readmission_rates["Score"].str.contains("Not Available")]),
      "out of a possible ", total_ccns, "facilities.")

275 out of a possible  1192 facilities.


I feel comfortable just dropping those facilities; it does account for about 23% of the dataset, which is a lot, but it won't kill us.

In [100]:
#Drop providers without readmission rate
readmission_rates = readmission_rates.loc[~(readmission_rates["Score"].str.contains("Not Available"))]

In [101]:
#Convert readmission rates to numeric values
readmission_rates["Score"] = readmission_rates["Score"].apply(pd.to_numeric)
readmission_rates.describe()

,Zip Code,CMS Region,Score
count,2109.000000,2109.000000,2109.000000
mean,51922.667141,5.240398,251.521679
std,26988.402353,2.173359,412.269217
min,674.000000,1.000000,5.000000
25%,30033.000000,4.000000,13.360000
50%,50309.000000,5.000000,17.000000
75%,75482.000000,6.000000,329.000000
max,99519.000000,10.000000,6393.000000


So at each end of the spectrum, we have a minimum readmission rate of 10.9% and a maximum of 15.84%. Is this in any way connected to the number of patients that each facility has to care for?

In [150]:
import numpy as np

#Reformat provider_data so that each row is a facility and its constituent measures
# ccndict = {"CMS Certification Number (CCN)" : [ccn for ccn in provider_data["CMS Certification Number (CCN)"].unique()]}
datadict = dict([(measure, pd.Series([])) for measure in provider_data["Measure Code"].unique()])
datadict = {**datadict,**{"CMS Certification Number (CCN)" : pd.Series([])}}


#Really inefficient, but there's not a great way of doing this as far as I can see
facility_data = provider_data.groupby(provider_data["CMS Certification Number (CCN)"])
for name, group in facility_data:
    for measure in group["Measure Code"].unique():
        score = group.loc[group["Measure Code"] == measure]["Score"]
        if score.iloc[0].lower() == "not available":
            score = pd.Series([np.nan])
        datadict[measure] = datadict[measure].append(score, ignore_index=True)
        datadict["CMS Certification Number (CCN)"] = datadict["CMS Certification Number (CCN)"].append(pd.Series(name), ignore_index=True)


In [151]:
test_one = pd.Series([1])
test_two = pd.Series([2])
test_one = test_one.append(test_two, ignore_index=True)
test_one

0    1
1    2
dtype: int64

In [152]:
facility_data = pd.DataFrame(datadict)
facility_data

,CMS Certification Number (CCN),I_001_01_ADJ_RATE,I_001_01_DENOMINATOR,I_002_01_DENOMINATOR,I_002_01_OBS_RATE,I_006_01_CI_LOWER,I_006_01_CI_UPPER,I_006_01_COMP_PERF,I_006_01_DOPC_DAYS,I_006_01_ELIGCASES,...,I_014_01_SIR,I_015_01_CI_LOWER,I_015_01_CI_UPPER,I_015_01_COMP_PERF,I_015_01_DOPC_DAYS,I_015_01_ELIGCASES,I_015_01_NUMERATOR,I_015_01_SIR,I_016_01_DENOMINATOR,I_016_01_OBS_RATE
0,013025,0.7,2259,NaN,NaN,0.151,2.972,No Different than the National Benchmark,1568,2.223,...,NaN,0.712,1.855,No Different than the National Benchmark,31803,14.377,17,1.182,NaN,NaN
1,013025,0.8,1467,NaN,NaN,NaN,1.304,No Different than the National Benchmark,2127,2.297,...,NaN,1.390,3.287,Worse than the National Benchmark,21566,9.600,21,2.188,NaN,NaN
2,013025,0.1,1322,NaN,NaN,NaN,2.496,No Different than the National Benchmark,1111,1.200,...,NaN,0.085,0.914,Better than the National Benchmark,23963,8.934,3,0.336,NaN,NaN
3,013025,0.1,1277,NaN,NaN,NaN,1.597,No Different than the National Benchmark,1212,1.876,...,NaN,1.044,2.897,Worse than the National Benchmark,18117,8.349,15,1.797,NaN,NaN
4,013025,0.6,943,NaN,NaN,0.242,4.764,No Different than the National Benchmark,1285,1.387,...,NaN,0.509,2.610,No Different than the National Benchmark,14187,4.782,6,1.255,NaN,NaN
5,013025,0.5,1147,NaN,NaN,0.258,5.079,No Different than the National Benchmark,1205,1.301,...,NaN,0.328,1.682,No Different than the National Benchmark,16208,7.419,6,0.809,NaN,NaN
6,013025,0.0,234,NaN,NaN,NaN,NaN,NaN,368,0.700,...,NaN,2.383,10.778,Worse than the National Benchmark,4343,1.285,7,5.449,NaN,NaN
7,013025,0.7,97,NaN,NaN,NaN,NaN,NaN,48,0.069,...,NaN,NaN,NaN,NaN,1657,0.658,0,NaN,NaN,NaN
8,013025,0.5,490,NaN,NaN,0.692,7.402,No Different than the National Benchmark,533,1.103,...,NaN,0.159,1.211,No Different than the National Benchmark,15618,7.970,4,0.502,NaN,NaN
9,013025,1.2,486,NaN,NaN,1.649,7.456,Worse than the National Benchmark,976,1.857,...,NaN,1.293,5.288,Worse than the National Benchmark,9826,2.873,8,2.785,NaN,NaN
